# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [6]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [2]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [15]:
from scipy.stats import ttest_1samp

salari_ref = 30   # La media del salario/hora es #30/h
# Hipotesis nula: La media del salario por hora de los trabajadores es igual a #30/h

salaries_clean = salaries['Hourly Rate'].dropna()  # Elimino los NaN
salario_hora = ttest_1samp(salaries_clean, salari_ref)
print(salario_hora)

# El resultado del valor t indica que el salario de referencia està a más de 20 unidades de desviación estándard de la media de la muestra, lo que sugiere una diferencia considerable entre los dos valores.
# De igual manera, el p-value es extremadamente pequeño, muy por debajo del 0.01. Ésto indica que la probabilidad de observar esta diferencia por puro azar es pràcticamente nula.
# Por consiguiente, hay suficiente evidència para rechazar la hipòtesi nula i concluir que el salario por hora medio es muy diferente de los $30/hora.

TtestResult(statistic=np.float64(20.6198057854942), pvalue=np.float64(4.3230240486229894e-92), df=np.int64(8021))


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [21]:
# Nivell de confiança
confidence = 0.95
# Graus de llibertat: nombre de mostres no NaN menys 1
df = len(salaries_clean) - 1
# Mitjana de la mostra
loc = salaries_clean.mean()
# valor de l'escala
scale = sem(salaries_clean)
print(f"{alpha=}, {df=}, {loc=}, {scale=}")
confidence_interval = t.interval(
    confidence=confidence,
    df=df,
    loc=loc,
    scale=scale)
print(f"Intérvalo de confianza {confidence_interval}")
# Por el resultado, podemos ver que hay un 95% de confianza de que el salario por hora medio esté entre 32,52 y 33,05,
# y por tanto, el valor de referencia ($30/h) quedaría fuera del intérvalo. Un resultat coherent amb la prova de la hipòtesi,
# que refuerza la evidencia en contra de la hipótesis nula.

alpha=0.95, df=8021, loc=np.float64(32.78855771628024), scale=np.float64(0.1352368565101596)
Intérvalo de confianza (np.float64(32.52345834488425), np.float64(33.05365708767623))


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [22]:
# Dado que no dispongo de la información de ciudad en el dataset, decido hacer el cálculo sobre la columna Department con el valor POLICE.
# Así, el enunciado quedaría: utiliza la función proportions_ztest de statsmodels para realizar una prueba de hipótesis que determinará si el
# número de trabajadores por hora del depatamento de policía es significativamente diferente del 25% en el nivel de confianza del 95%.

# Con todo, esta interpretación no me deja del todo satisfecho. Al parecer, no se pide tanto sobre el salario por hora sinó por el número de
# trabajadores por hora. Me temo que no sé interpretar esta petición y seguiré trabajando sobre la proporción de trabajadores que tienen Hourly Rate.

In [27]:
from statsmodels.stats.proportion import proportions_ztest

# La hipòtesi nul·la és que el nombre de treballadors 

# Treballadors del departament de policia
treballadors_amb_hourly_rate = salaries[salaries['Hourly Rate'].notna()]
policies = treballadors_amb_hourly_rate[treballadors_amb_hourly_rate['Department'] == 'POLICE']

# Proporció de referència
proporcio = 0.25

# Realitzem les proves de proporcions
stat, p_value = proportions_ztest(len(policies), len(treballadors_amb_hourly_rate), value=proporcio, alternative='two-sided')

print(f"{stat=}, {p_value=}")

# Observamos que el estadístico z indica una distancia muy grande, en número de desviaciones estándar, entre la proporción de referencia (0.25) y la proporción observada.  
# Además, el valor p es 0.0, un valor que no deja margen para no rechazar la hipótesis nula.  
# Por lo tanto, estos resultados nos permiten concluir que el número de trabajadores del departamento de policía es significativamente diferente al 25%, con un nivel de confianza del 95%.

stat=np.float64(-631.4261888936179), p_value=np.float64(0.0)
